In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
aisles = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/departments.csv')
orders = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/products.csv')

In [4]:
#op = pd.read_csv('order_products__train.csv')
#opp = pd.read_csv('order_products__prior.csv')
#aisles = pd.read_csv('aisles.csv')
#departments = pd.read_csv('departments.csv')
#orders = pd.read_csv('orders.csv')
#products = pd.read_csv('products.csv')

In [5]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
# order_products Tabellen laden (die _prior und _train-Tabellen können für unsere Zwecke gemerged werden)
order_products = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/order_products__prior.csv', usecols=['order_id', 'product_id', 'reordered'])
order_products_train = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/order_products__train.csv', usecols=['order_id', 'product_id', 'reordered'])
order_products.append(order_products_train)

,order_id,product_id,reordered
0,2,33120,1
1,2,28985,1
2,2,9327,0
3,2,45918,1
4,2,30035,0
...,...,...,...
1384612,3421063,14233,1
1384613,3421063,35548,1
1384614,3421070,35951,1
1384615,3421070,16953,1


In [ ]:
## Eine Spalte 'contains_bananas' zum 'orders'-DataFrame hinzufügen:

In [8]:
products_filter = products[products['product_name'].isin(["Bag of Organic Bananas","Organic Banana"])]
products_filter = products_filter['product_id']
order_products_with_bananas = order_products[order_products['product_id'].isin(products_filter)]

In [9]:
orders_with_bananas = order_products_with_bananas['order_id'].unique() # IDs of all orders with bananas

In [10]:
orders['contains_bananas'] = 0
orders.loc[orders['order_id'].isin(orders_with_bananas),'contains_bananas'] = 1

## Create dataframe or array X of inputs and one of outputs Y 

#### Eine Spalte 'previous_order_contains_bananas' zum 'orders' DataFrame hinzufügen

In [11]:
h = orders[['order_id','user_id','order_number','contains_bananas']]
h2 = h.copy()
h2['on_orig']=h2['order_number']
h2['order_number'] -= 1
order_pairs = h.merge(h2, on=['user_id','order_number']) # The columns named *_x are from the previous order, the ones from *_y from the current order


In [ ]:
# Daraus X und Y extrahieren

In [12]:
X = order_pairs['contains_bananas_x']
Y = order_pairs['contains_bananas_y']

In [13]:
def copy_input_model(X):
    return X 
def always_say_no_model(X):
    return np.zeros(X.shape[0])
def always_say_yes_model(X):
    return np.ones(X.shape[0])
def random_model(X, prob=0.5):
    return (np.random.rand(X.shape[0])<prob)*1 # the "*1" converts the array of booleans to one of 0/1s


In [14]:
def accuracy(out, true_out):
    return sum(true_out==out)/len(out)
def precision(out, true_out):
    return sum((out==1) & (true_out==1))/sum(out==1)
def recall(out, true_out):
    return sum((out==1) & (true_out==1))/sum(true_out==1)
def f1(out, true_out):
    return 2/(1/precision(out,true_out)+1/recall(out,true_out))
def kosten(out, true_out):
    return sum((out==1) & (true_out==0))*0.11+sum((out==0) & (true_out==1))*0.89


In [15]:
out = copy_input_model(X)
print("Accuracy: ",accuracy(out,Y))
print("Precision: ",precision(out,Y))
print("Recall: ",recall(out,Y))
print("F1-score: ",f1(out,Y))
print("Kosten: ",kosten(out,Y))


Accuracy:  0.8848819580487447
Precision:  0.509921507616752
Recall:  0.536751458391206
F1-score:  0.5229926094723518
Kosten:  177288.68000000002


In [16]:
out = always_say_no_model(X)
print("Accuracy: ",accuracy(out,Y))
# print("Precision: ",precision(out,Y))
print("Recall: ",recall(out,Y))
# print("F1-score: ",f1(out,Y))
print("Kosten: ",kosten(out,Y))

Accuracy:  0.8824261852875105
Recall:  0.0
Kosten:  336406.65


In [17]:
out = always_say_yes_model(X)
print("Accuracy: ",accuracy(out,Y))
print("Precision: ",precision(out,Y))
print("Recall: ",recall(out,Y))
print("F1-score: ",f1(out,Y))
print("Kosten: ",kosten(out,Y))

Accuracy:  0.11757381471248951
Precision:  0.11757381471248951
Recall:  1.0
F1-score:  0.21040903636908656
Kosten:  312057.79


#### Im Folgenden werden die Werte der *Konfusionsmatrix* berechnet:

In [18]:
# prev and subsequent order contain bananas 
b_b = len(order_pairs[(order_pairs['contains_bananas_x'] == 1 ) & (order_pairs['contains_bananas_y'] == 1 )].index)
b_b

202884

In [19]:
# prev order contains no bananas, subsequent does
nb_b = len(order_pairs[(order_pairs['contains_bananas_x'] == 0 ) & (order_pairs['contains_bananas_y'] == 1 )].index)
nb_b

175101

In [20]:
# prev order contains bananas, subsequent does not
b_nb = len(order_pairs[(order_pairs['contains_bananas_x'] == 1 ) & (order_pairs['contains_bananas_y'] == 0 )].index)
b_nb

194989

In [21]:
# neither previous nor subsequemt order contain bananas
nb_nb = len(order_pairs[(order_pairs['contains_bananas_x'] == 0 ) & (order_pairs['contains_bananas_y'] == 0 )].index)
nb_nb

2641900

In [22]:
total_orders = b_b + nb_b + b_nb + nb_nb
total_orders

3214874

### Antworten auf 3a, 3b, 3c

In [23]:
# 3a: Wie wahrscheinlich ist es, dass die nächste Bestellung eines Users 'organic bananas' enthält wenn die vorherige Bestellung organic bananas enthalten hat.
b_b / (b_b + b_nb)

0.509921507616752

In [24]:
# 3b: Wie wahrscheinlich ist es, dass die nächste Bestellung eines Users 'organic bananas' enthält wenn die vorherige Bestellung keine organic bananas enthalten hat.
nb_b / (nb_b + nb_nb)

0.06215865738066831

In [25]:
# 3c: Wie wahrscheinlich ist es, dass die nächste Bestellung eines Users 'organic bananas' enthält (egal was vorher war)
(nb_b + b_b) / total_orders

0.11757381471248951